In [1]:
from detectron2.config import LazyConfig, instantiate

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from detectron2.engine import default_setup
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

In [3]:
cfg = LazyConfig.load("/data/ephemeral/home/level2-objectdetection-cv-01/detectron2/projects/ViTDet/configs/COCO/cascade_mask_rcnn_vitdet_b_100ep.py")
default_setup(cfg, "")

[10/21 11:48:33 detectron2]: Rank of current process: 0. World size: 1
[10/21 11:48:34 detectron2]: Environment info:
-------------------------------  ----------------------------------------------------------------------------
sys.platform                     linux
Python                           3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
numpy                            1.22.4
detectron2                       0.6 @/data/ephemeral/home/level2-objectdetection-cv-01/detectron2/detectron2
Compiler                         GCC 9.4
CUDA compiler                    not available
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          1.12.1+cu116 @/opt/conda/lib/python3.10/site-packages/torch
PyTorch debug build              False
torch._C._GLIBCXX_USE_CXX11_ABI  False
GPU available                    Yes
GPU 0                            Tesla V100-SXM2-32GB (arch=7.0)
Driver version                   535.161.08
CUDA_HOME                        None - invalid!
Pi

In [4]:
try:
    register_coco_instances('coco_trash_train', {}, '/data/ephemeral/home/dataset/train_80.json', '/data/ephemeral/home/dataset')
except AssertionError:
    pass
try:
    register_coco_instances('coco_trash_test', {}, '/data/ephemeral/home/dataset/val_20.json', '/data/ephemeral/home/dataset')
except AssertionError:
    pass

MetadataCatalog.get('coco_trash_train').thing_classes = ["General trash", "Paper", "Paper pack", "Metal", 
                                                        "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]


In [5]:
model = instantiate(cfg.model)
model.to(cfg.train.device)
train_loader = instantiate(cfg.dataloader.train)

/opt/conda/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


[10/21 11:48:38 d2.data.datasets.coco]: Loaded 3914 images in COCO format from /data/ephemeral/home/dataset/train_80.json
[10/21 11:48:39 d2.data.build]: Removed 0 images with no usable annotations. 3914 images left.
[10/21 11:48:39 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category   | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
| General trash | 3161         |    Paper    | 5115         | Paper pack | 706          |
|     Metal     | 769          |    Glass    | 835          |  Plastic   | 2350         |
|   Styrofoam   | 1026         | Plastic bag | 4151         |  Battery   | 143          |
|   Clothing    | 377          |             |              |            |              |
|     total     | 18633        |             |              |            |              |
[10/21 11:48:39 d2.data.common]: Serializing the dataset using: <class

In [6]:
model.backbone

SimpleFeaturePyramid(
  (simfp_2): Sequential(
    (0): ConvTranspose2d(768, 384, kernel_size=(2, 2), stride=(2, 2))
    (1): LayerNorm()
    (2): GELU(approximate=none)
    (3): ConvTranspose2d(384, 192, kernel_size=(2, 2), stride=(2, 2))
    (4): Conv2d(
      192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): LayerNorm()
    )
    (5): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): LayerNorm()
    )
  )
  (simfp_3): Sequential(
    (0): ConvTranspose2d(768, 384, kernel_size=(2, 2), stride=(2, 2))
    (1): Conv2d(
      384, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): LayerNorm()
    )
    (2): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): LayerNorm()
    )
  )
  (simfp_4): Sequential(
    (0): Conv2d(
      768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): LayerNorm()
    )
    (1): Conv2d(
      256, 256, kernel_size=

In [7]:
for batch in train_loader:
    break

In [8]:
image = model.preprocess_image(batch)

In [9]:
image.tensor.shape

torch.Size([2, 3, 1024, 1024])

In [10]:
test = model.backbone(image.tensor)

In [11]:
batch[0]['instances']

Instances(num_instances=1, image_height=1024, image_width=1024, fields=[gt_boxes: Boxes(tensor([[156.8000, 290.3000, 751.1000, 770.3000]])), gt_classes: tensor([5])])

In [12]:
for i in test.keys():
    print(i, " : ",test[i].shape)

p2  :  torch.Size([2, 256, 256, 256])
p3  :  torch.Size([2, 256, 128, 128])
p4  :  torch.Size([2, 256, 64, 64])
p5  :  torch.Size([2, 256, 32, 32])
p6  :  torch.Size([2, 256, 16, 16])


In [13]:
test['p6'].shape

torch.Size([2, 256, 16, 16])

In [14]:
gt_instances = [x["instances"].to(model.device) for x in batch]

In [15]:
gt_instances

[Instances(num_instances=1, image_height=1024, image_width=1024, fields=[gt_boxes: Boxes(tensor([[156.8000, 290.3000, 751.1000, 770.3000]], device='cuda:0')), gt_classes: tensor([5], device='cuda:0')]),
 Instances(num_instances=1, image_height=1024, image_width=1024, fields=[gt_boxes: Boxes(tensor([[202., 128., 647., 815.]], device='cuda:0')), gt_classes: tensor([2], device='cuda:0')])]

In [16]:
model.proposal_generator.in_features

['p2', 'p3', 'p4', 'p5', 'p6']

In [17]:
features = [test[f] for f in model.proposal_generator.in_features]

In [18]:
model.proposal_generator.in_features

['p2', 'p3', 'p4', 'p5', 'p6']

In [19]:
model.roi_heads.in_features

['p2', 'p3', 'p4', 'p5']

In [20]:
anchors = model.proposal_generator.anchor_generator(features)

/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [21]:
for i in anchors:
    print( " : ",i.tensor.shape)

 :  torch.Size([196608, 4])
 :  torch.Size([49152, 4])
 :  torch.Size([12288, 4])
 :  torch.Size([3072, 4])
 :  torch.Size([768, 4])


In [22]:
# model.proposal_generator(image, test, gt_instances)

In [23]:
pred_objectness_logits, pred_anchor_deltas = model.proposal_generator.rpn_head(features)

In [24]:
pred_objectness_logits[0].shape

torch.Size([2, 3, 256, 256])

In [25]:
pred_objectness_logits[1].shape

torch.Size([2, 3, 128, 128])

In [26]:
pred_objectness_logits = [
    # (N, A, Hi, Wi) -> (N, Hi, Wi, A) -> (N, Hi*Wi*A)
    score.permute(0, 2, 3, 1).flatten(1)
    for score in pred_objectness_logits
]


In [27]:
pred_anchor_deltas = [
# (N, A*B, Hi, Wi) -> (N, A, B, Hi, Wi) -> (N, Hi, Wi, A, B) -> (N, Hi*Wi*A, B)
x.view(x.shape[0], -1, model.proposal_generator.anchor_generator.box_dim, x.shape[-2], x.shape[-1])
.permute(0, 3, 4, 1, 2)
.flatten(1, -2)
for x in pred_anchor_deltas
]



In [28]:
pred_anchor_deltas[0].shape

torch.Size([2, 196608, 4])

In [29]:
# gt_labels, gt_boxes = model.proposal_generator.label_and_sample_anchors(anchors, gt_instances)

In [30]:
image.image_sizes

[(1024, 1024), (1024, 1024)]

In [31]:
proposals = model.proposal_generator.predict_proposals(
    anchors, pred_objectness_logits, pred_anchor_deltas, image.image_sizes
)


In [32]:
proposals[0]

Instances(num_instances=1000, image_height=1024, image_width=1024, fields=[proposal_boxes: Boxes(tensor([[550.5684, 680.6083, 615.2935, 745.9509],
        [935.3654, 232.6556, 967.7941, 264.0651],
        [923.0459, 243.5042, 954.9937, 275.3983],
        ...,
        [859.0336, 708.1061, 891.4944, 739.9067],
        [883.4612, 284.5491, 915.4044, 316.4648],
        [807.5496, 840.6591, 872.2675, 904.8127]], device='cuda:0')), objectness_logits: tensor([0.0551, 0.0531, 0.0519, 0.0503, 0.0500, 0.0487, 0.0487, 0.0482, 0.0481,
        0.0480, 0.0472, 0.0471, 0.0471, 0.0467, 0.0465, 0.0464, 0.0463, 0.0463,
        0.0462, 0.0459, 0.0453, 0.0450, 0.0448, 0.0448, 0.0448, 0.0447, 0.0446,
        0.0444, 0.0444, 0.0444, 0.0443, 0.0442, 0.0441, 0.0440, 0.0440, 0.0439,
        0.0436, 0.0436, 0.0435, 0.0435, 0.0433, 0.0432, 0.0431, 0.0431, 0.0429,
        0.0428, 0.0428, 0.0426, 0.0426, 0.0426, 0.0425, 0.0423, 0.0423, 0.0423,
        0.0423, 0.0422, 0.0421, 0.0421, 0.0421, 0.0420, 0.0419, 0.0419,

In [33]:
# model.roi_heads.label_and_sample_proposals(proposals,gt_instances)

In [34]:
model.roi_heads.proposal_append_gt

True

In [35]:
# proposals = model.roi_heads.add_ground_truth_to_proposals(gt_instances,proposals)

In [36]:
from detectron2.modeling.proposal_generator.proposal_utils import add_ground_truth_to_proposals


In [37]:
proposals = add_ground_truth_to_proposals(gt_instances,proposals)

In [38]:
proposals

[Instances(num_instances=1001, image_height=1024, image_width=1024, fields=[proposal_boxes: Boxes(tensor([[550.5684, 680.6083, 615.2935, 745.9509],
         [935.3654, 232.6556, 967.7941, 264.0651],
         [923.0459, 243.5042, 954.9937, 275.3983],
         ...,
         [883.4612, 284.5491, 915.4044, 316.4648],
         [807.5496, 840.6591, 872.2675, 904.8127],
         [156.8000, 290.3000, 751.1000, 770.3000]], device='cuda:0')), objectness_logits: tensor([ 0.0551,  0.0531,  0.0519,  ...,  0.0333,  0.0333, 23.0259],
        device='cuda:0')]),
 Instances(num_instances=1001, image_height=1024, image_width=1024, fields=[proposal_boxes: Boxes(tensor([[ 116.6944,  501.0135,  161.3426,  523.3392],
         [ 727.5382, 1000.6201,  760.1986, 1024.0000],
         [ 919.3065,  824.3824,  951.7719,  856.6013],
         ...,
         [ 975.1458,  403.9811, 1007.5734,  436.6864],
         [ 479.5677,  947.8969,  512.1809,  980.1924],
         [ 202.0000,  128.0000,  647.0000,  815.0000]], devic

In [39]:
from detectron2.structures import  pairwise_iou


In [40]:
proposals_with_gt = []

num_fg_samples = []
num_bg_samples = []


for proposals_per_image, targets_per_image in zip(proposals, gt_instances):
    has_gt = len(targets_per_image) > 0
    match_quality_matrix = pairwise_iou(
        targets_per_image.gt_boxes, proposals_per_image.proposal_boxes
    )
    matched_idxs, matched_labels = model.roi_heads.proposal_matcher(match_quality_matrix)
    sampled_idxs, gt_classes = model.roi_heads._sample_proposals(
        matched_idxs, matched_labels, targets_per_image.gt_classes
    )

    # Set target attributes of the sampled proposals:
    proposals_per_image = proposals_per_image[sampled_idxs]
    proposals_per_image.gt_classes = gt_classes

    if has_gt:
        sampled_targets = matched_idxs[sampled_idxs]
        # We index all the attributes of targets that start with "gt_"
        # and have not been added to proposals yet (="gt_classes").
        # NOTE: here the indexing waste some compute, because heads
        # like masks, keypoints, etc, will filter the proposals again,
        # (by foreground/background, or number of keypoints in the image, etc)
        # so we essentially index the data twice.
        for trg_name, trg_value in targets_per_image.get_fields().items():
            if trg_name.startswith("gt_") and not proposals_per_image.has(trg_name):
                proposals_per_image.set(trg_name, trg_value[sampled_targets])
    # If no GT is given in the image, we don't know what a dummy gt value can be.
    # Therefore the returned proposals won't have any gt_* fields, except for a
    # gt_classes full of background label.

    num_bg_samples.append((gt_classes == model.roi_heads.num_classes).sum().item())
    num_fg_samples.append(gt_classes.numel() - num_bg_samples[-1])
    proposals_with_gt.append(proposals_per_image)


In [41]:
proposals_with_gt # label_and_sample_proposals output 

[Instances(num_instances=512, image_height=1024, image_width=1024, fields=[proposal_boxes: Boxes(tensor([[ 156.8000,  290.3000,  751.1000,  770.3000],
         [ 983.9510,  456.3400, 1016.5684,  487.5888],
         [ 715.0809,  532.2385,  747.4502,  563.6819],
         ...,
         [  39.3918,  440.4337,   71.6111,  471.7375],
         [ 351.5990,  713.0428,  383.7295,  744.7601],
         [ 551.4412,  840.4567,  616.1255,  904.6942]], device='cuda:0')), objectness_logits: tensor([23.0259,  0.0340,  0.0353,  0.0334,  0.0333,  0.0388,  0.0402,  0.0333,
          0.0348,  0.0345,  0.0343,  0.0419,  0.0337,  0.0351,  0.0421,  0.0376,
          0.0347,  0.0401,  0.0379,  0.0345,  0.0333,  0.0354,  0.0362,  0.0335,
          0.0369,  0.0359,  0.0344,  0.0393,  0.0339,  0.0362,  0.0359,  0.0345,
          0.0465,  0.0340,  0.0396,  0.0374,  0.0358,  0.0383,  0.0343,  0.0340,
          0.0353,  0.0345,  0.0376,  0.0335,  0.0336,  0.0372,  0.0337,  0.0333,
          0.0334,  0.0342,  0.0336, 

In [42]:
model.roi_heads.box_in_features

['p2', 'p3', 'p4', 'p5']

In [43]:
len(features)

5

In [44]:
head_outputs = []  # (predictor, predictions, proposals)
prev_pred_boxes = None
image_sizes = [x.image_size for x in proposals]


In [45]:
model.roi_heads.num_cascade_stages

3

In [46]:
from detectron2.structures import Boxes
import torch

In [47]:
features = [test[f] for f in model.roi_heads.box_in_features]

In [48]:
len(features)

4

In [49]:
def _match_and_label_boxes(roi,proposals, stage, targets):
    num_fg_samples, num_bg_samples = [], []
    for proposals_per_image, targets_per_image in zip(proposals, targets):
        match_quality_matrix = pairwise_iou(
            targets_per_image.gt_boxes, proposals_per_image.proposal_boxes
        )
        # proposal_labels are 0 or 1
        matched_idxs, proposal_labels = roi.proposal_matchers[stage](match_quality_matrix)
        if len(targets_per_image) > 0:
            gt_classes = targets_per_image.gt_classes[matched_idxs]
            # Label unmatched proposals (0 label from matcher) as background (label=num_classes)
            gt_classes[proposal_labels == 0] = roi.num_classes
            gt_boxes = targets_per_image.gt_boxes[matched_idxs]
        else:
            gt_classes = torch.zeros_like(matched_idxs) + roi.num_classes
            gt_boxes = Boxes(
                targets_per_image.gt_boxes.tensor.new_zeros((len(proposals_per_image), 4))
            )

        proposals_per_image.gt_classes = gt_classes
        proposals_per_image.gt_boxes = gt_boxes
        # proposals_per_image.set("gt_classes", gt_classes)
        # proposals_per_image.set("gt_boxes", gt_boxes)


        num_fg_samples.append((proposal_labels == 1).sum().item())
        num_bg_samples.append(proposal_labels.numel() - num_fg_samples[-1])


    return proposals

In [50]:
proposals[0].proposal_boxes.tensor.shape

torch.Size([1001, 4])

In [51]:
box_features = model.roi_heads.box_pooler(features, [x.proposal_boxes for x in proposals])

In [52]:
box_features.shape

torch.Size([2002, 256, 7, 7])

In [53]:
for k in range(model.roi_heads.num_cascade_stages):
    if k > 0:
        # The output boxes of the previous stage are used to create the input
        # proposals of the next stage.
        proposals = model.roi_heads._create_proposals_from_boxes(prev_pred_boxes, image_sizes)
        # if self.training:
        # proposals = model.roi_heads._match_and_label_boxes(proposals, k, gt_instances)
        proposals = _match_and_label_boxes(model.roi_heads,proposals, k, gt_instances)

    predictions = model.roi_heads._run_stage(features, proposals, k)
    prev_pred_boxes = model.roi_heads.box_predictor[k].predict_boxes(predictions, proposals)
    head_outputs.append((model.roi_heads.box_predictor[k], predictions, proposals))


In [54]:
model.roi_heads.num_classes

80

In [55]:
predictions[0].shape

torch.Size([2002, 81])

In [56]:
proposals[0].gt_classes


tensor([80, 80, 80,  ..., 80, 80,  5], device='cuda:0')

In [57]:
len(proposals)

2

In [58]:
# proposals[0].gt_classes
# head_outputs[0][2][0].gt_classes

In [59]:
# head_outputs[0][2][0].gt_classes

In [60]:
# losses = {}
# for stage, (predictor, predictions, proposals) in enumerate(head_outputs):
#     break
    # predictor.losses(predictions, proposals)

In [61]:
proposals

[Instances(num_instances=1001, image_height=1024, image_width=1024, fields=[proposal_boxes: Boxes(tensor([[550.5065, 680.7580, 615.4693, 745.9525],
         [935.3456, 232.8400, 967.8932, 263.9976],
         [923.0463, 243.6731, 955.0397, 275.3087],
         ...,
         [883.5032, 284.6774, 915.4601, 316.4305],
         [807.3915, 840.8939, 872.5300, 904.6993],
         [157.7542, 292.7725, 751.0677, 768.8397]], device='cuda:0')), gt_classes: tensor([80, 80, 80,  ..., 80, 80,  5], device='cuda:0'), gt_boxes: Boxes(tensor([[156.8000, 290.3000, 751.1000, 770.3000],
         [156.8000, 290.3000, 751.1000, 770.3000],
         [156.8000, 290.3000, 751.1000, 770.3000],
         ...,
         [156.8000, 290.3000, 751.1000, 770.3000],
         [156.8000, 290.3000, 751.1000, 770.3000],
         [156.8000, 290.3000, 751.1000, 770.3000]], device='cuda:0'))]),
 Instances(num_instances=1001, image_height=1024, image_width=1024, fields=[proposal_boxes: Boxes(tensor([[ 116.7133,  501.1270,  161.351

In [62]:
scores, proposal_deltas = predictions

In [63]:
scores.shape

torch.Size([2002, 81])

In [64]:
proposal_deltas.shape

torch.Size([2002, 4])

In [65]:
from detectron2.layers import cat

In [66]:
proposals[0].gt_classes

tensor([80, 80, 80,  ..., 80, 80,  5], device='cuda:0')

In [67]:
gt_classes = (
    cat([p.gt_classes for p in proposals], dim=0) if len(proposals) else torch.empty(0)
)


In [68]:
gt_classes

tensor([80, 80, 80,  ..., 80, 80,  2], device='cuda:0')

In [69]:
proposal_boxes = cat([p.proposal_boxes.tensor for p in proposals], dim=0) 

In [70]:
proposal_boxes.shape

torch.Size([2002, 4])

In [71]:
gt_boxes = cat(
    [(p.gt_boxes if p.has("gt_boxes") else p.proposal_boxes).tensor for p in proposals],
    dim=0,
)


In [72]:
model.roi_heads.box_predictor[k].box_reg_loss(
                proposal_boxes, gt_boxes, proposal_deltas, gt_classes
            )

tensor(0.0002, device='cuda:0', grad_fn=<DivBackward0>)

In [73]:
from detectron2.layers import nonzero_tuple


In [74]:
box_dim = proposal_boxes.shape[1]  # 4 or 5
# Regression loss is only computed for foreground proposals (those matched to a GT)
fg_inds = nonzero_tuple((gt_classes >= 0) & (gt_classes < 10))[0]


In [75]:
# proposal_deltas.shape[1] == box_dim
fg_pred_deltas = proposal_deltas[fg_inds]

In [76]:

fg_pred_deltas.shape

torch.Size([2, 4])

In [77]:
fg_pred_deltas

tensor([[-5.4823e-05,  5.7946e-03, -4.0293e-04, -1.4985e-02],
        [ 7.0027e-03,  7.1271e-04, -1.4262e-03, -1.2130e-02]], device='cuda:0',
       grad_fn=<IndexBackward0>)

In [78]:
box_features[fg_inds].shape

torch.Size([2, 256, 7, 7])

In [79]:
gt_boxes[fg_inds].shape

torch.Size([2, 4])

In [80]:
# model.roi_heads.box_predictor[k].box2box_transform.apply_deltas

In [81]:
# isinstance(anchors[0], Boxes)
anchors = type(anchors[0]).cat(anchors).tensor

In [82]:
fg_pred_deltas.shape

torch.Size([2, 4])

In [83]:
cat([proposal_boxes[fg_inds]]).shape

torch.Size([2, 4])

In [84]:
cat([fg_pred_deltas.unsqueeze(0)], dim=1).shape

torch.Size([1, 2, 4])

In [85]:
pred_boxes = [
    model.roi_heads.box_predictor[1].box2box_transform.apply_deltas(k, cat([proposal_boxes[fg_inds]])) for k in cat([fg_pred_deltas.unsqueeze(0)], dim=1)
]


In [86]:
pred_boxes

[tensor([[157.7645, 293.2669, 751.0542, 768.6212],
         [202.1630, 132.1035, 646.6474, 811.4620]], device='cuda:0',
        grad_fn=<ReshapeAliasBackward0>)]

In [87]:
# pred_boxes = [
#     model.roi_heads.box_predictor[1].box2box_transform.apply_deltas(k, proposal_boxes[fg_inds]) for k in cat([fg_pred_deltas.unsqueeze(0)], dim=1)
# ]


In [88]:
from detectron2.layers import ciou_loss

In [89]:
fg_inds

tensor([1000, 2001], device='cuda:0')

In [90]:
# torch.stack(pred_boxes)[fg_inds]

In [91]:
# loss_box_reg = ciou_loss(
#     torch.stack(pred_boxes)[fg_inds], torch.stack(gt_boxes)[fg_inds], reduction="sum"
# )


In [92]:
fg_pred_deltas

tensor([[-5.4823e-05,  5.7946e-03, -4.0293e-04, -1.4985e-02],
        [ 7.0027e-03,  7.1271e-04, -1.4262e-03, -1.2130e-02]], device='cuda:0',
       grad_fn=<IndexBackward0>)

In [93]:
gt_boxes[fg_inds]

tensor([[156.8000, 290.3000, 751.1000, 770.3000],
        [202.0000, 128.0000, 647.0000, 815.0000]], device='cuda:0')

In [94]:
pred_box = model.roi_heads.box_predictor[1].box2box_transform.apply_deltas(fg_pred_deltas, anchors[fg_inds])

In [95]:
pred_boxes

[tensor([[157.7645, 293.2669, 751.0542, 768.6212],
         [202.1630, 132.1035, 646.6474, 811.4620]], device='cuda:0',
        grad_fn=<ReshapeAliasBackward0>)]

In [96]:
loss_box_reg = ciou_loss(
    torch.stack(pred_boxes), torch.stack([gt_boxes[fg_inds]]), reduction=None
)


In [97]:
loss_box_reg.shape

torch.Size([1, 2])

In [98]:
nonzero_tuple((gt_classes >= 0) & (gt_classes < 10))

(tensor([1000, 2001], device='cuda:0'),)

In [99]:
nonzero_tuple((loss_box_reg.squeeze() < 0.009))[0]

tensor([], device='cuda:0', dtype=torch.int64)

In [100]:
contrast_id = nonzero_tuple((loss_box_reg.squeeze() < 0.009))[0]

In [101]:
gt_classes==0

tensor([False, False, False,  ..., False, False, False], device='cuda:0')

In [109]:
contrasive_id = nonzero_tuple((gt_classes ==1))

In [112]:
class_u = nonzero_tuple((gt_classes >= 0) & (gt_classes<10))[0] 

torch.Size([2, 256, 7, 7])

In [118]:
iou_s = nonzero_tuple((loss_box_reg.squeeze() < 1))

In [121]:
box_features[class_u][iou_s]

tensor([[[[ 0.5407,  0.8819,  0.4522,  ...,  0.5720,  0.6507,  0.0340],
          [ 0.5867,  0.3559,  0.1641,  ...,  0.1332,  0.0601, -0.3083],
          [ 0.3466,  0.3771,  0.2040,  ..., -0.1141, -0.4555, -0.5321],
          ...,
          [ 0.2350,  0.1901,  0.0882,  ..., -0.4161,  0.3196,  0.2546],
          [-0.2128,  0.1694,  0.2794,  ...,  0.3189,  0.0500,  0.1908],
          [-0.7755, -1.0153, -0.1351,  ...,  0.4753,  0.2535,  0.3023]],

         [[-1.0128, -0.6656, -0.6350,  ..., -0.0180,  0.0254, -0.4425],
          [-0.7938, -0.5283, -0.4104,  ...,  0.2164,  0.3022, -0.4729],
          [-0.5916, -0.4589,  0.0668,  ..., -0.0486, -0.1997, -0.2909],
          ...,
          [-0.4070,  0.0803,  0.0857,  ..., -0.7427, -0.5327, -0.2231],
          [-0.3752, -0.3980, -0.4383,  ..., -0.3977, -0.4066, -0.1119],
          [-0.4523, -0.4313, -0.1702,  ..., -0.1062, -0.1871, -0.1714]],

         [[ 0.3573,  0.4307,  0.1601,  ...,  0.0308,  0.0963,  0.2902],
          [ 0.3264,  0.1387,  

In [115]:
contrast_id

tensor([], device='cuda:0', dtype=torch.int64)

In [103]:
# box_features[contrasive_id][contrast_id]

In [104]:
# contrasive_id = nonzero_tuple((gt_classes ==0))

In [105]:
# fin = box_features[contrasive_id][contrast_id]

In [106]:
fin

NameError: name 'fin' is not defined

In [104]:
# box_features[fg_inds].shape

torch.Size([13, 256, 7, 7])